In [0]:
import pandas as pd
import numpy as np
import itertools
import os
os.chdir("/content/drive/My Drive/DMML Assignment")


In [2]:
ls

docword.enron.txt  kos_.txt    vocab.enron.txt
docword.kos.txt    nips_.txt   vocab.kos.txt
docword.nips.txt   output.txt  vocab.nips.txt


In [0]:
def get_transactions(data):
    data=pd.read_csv(data,sep=" ",skiprows=3,header=None)
    data.columns=['doc_id','word_id','occurences']
    data.drop('occurences',axis=1)
    data=data.groupby('doc_id')['word_id'].apply(tuple)
    return list(data)

   


In [0]:
def get_MS(data,alpha,L):
  
  support={}
  for t in data:
    for i in t:
      support[i]=(support.get(i,0)+1)
  
  MS ={k: max(L,(v*alpha)/len(data)) for k, v in support.items()}
  return MS



In [0]:
def init_pass(M,T):
    L = []
    for t in T:
        for i in t:
            sup_count[i] = sup_count.get(i,0)+1
    smallest_sup=0.095
    for m in M:
        if smallest_sup:
            if  MS[m] > smallest_sup:
               L.append(m)
        #elif m in sup_count and sup_count[m]/n >= MS[m]:
         #   L.append(m)
          #  smallest_sup = MS[m]
    #print(smallest_sup)

    return L


In [0]:
def level2_candidate_gen(L,phi):
    c = []
    for i,l in enumerate(L):
        if sup_count[l]/n >= MS[l]:
            for j in range (i+1,len(L)):
                if sup_count[L[j]]/n > MS[l] and abs(sup_count[L[j]]/n - sup_count[l]/n) <= phi:
                    c.append({'c':[l, L[j]],'count':0})
    return c

In [0]:
def MScandidate_gen(F, k, phi):
    c = []
    for index,f1 in enumerate(F):
        for j,f2 in enumerate(F[index+1:]):
            if set(f1[:-1]) == set(f2[:-1]) and abs(sup_count[f2[k-2]]/n - sup_count[f1[k-2]]/n) <= phi: 
                candidate = list(f1)
                candidate.append(f2[k-2])
                delete = False
                for s in list(itertools.combinations(candidate, k-1)):
                    if candidate[0] in s or MS[candidate[0]] == MS[candidate[1]]:
                        if list(s) not in F:
                            delete = True
                if not delete:
                    c.append({'c':candidate,'count':0})
    return c

In [0]:

T=get_transactions('docword.kos.txt')
MS=get_MS(T,alpha=0.6,L=0.090)
n=float(len(T))
F={}
sup_count={}
C={}
M=[]
for item, mis in sorted (MS.items(),key= lambda x:(x[1],int(x[0]))):
  M.append(item)
L=init_pass(M,T)
F[1]=[[L[0]]]
for l in L[1:]:
  if sup_count[l]/n>=MS[l]:
    F[1].append([l])

for k in range(2,7):
  if k == 2:
        C[k] = level2_candidate_gen(L,phi=0.3)
  else:
        C[k] = MScandidate_gen(F[k-1],k,phi=0.3)

  for c in C[k]:
        sub_found = False
        sub_count = True
        if k > 2:
            for c1 in C[k-1]:
                if set(c['c'][1:]) == set(c1['c']):
                    sub_found == True
                    if c1['count'] != 0:
                        sub_count = False
            if sub_count and not sub_found:
                C[k-1].append({'c':c['c'][1:], 'count':0})
        for t in T:
            if set(c['c']).issubset(t):
                c['count']+=1
            if k > 2 and sub_count:
                if set(c['c'][1:]).issubset(t):
                    C[k-1][-1]['count'] +=1

  if not C[k]:
    break

  F[k] = []
  for c in C[k]:
      if c['count']/n >= MS[c['c'][0]]:
          F[k].append(c['c'])

  if len(F[k]) < 2:
      break



In [0]:
def word_id_to_word(fset,vocabfile):
  wl=pd.read_csv(vocabfile,sep=" ",header=None)
  vocablist=wl[0].tolist()
  for key,value in fset.items():
    temp_1=[]
    for itemset in value:
      temp_2=[]
      for word_id in itemset:
        temp_2.append(vocablist[word_id - 1])
      temp_1.append(temp_2)
    fset[key]=temp_1
  return fset
    


In [0]:
F_=F.copy()

In [0]:
output=word_id_to_word(F_,'vocab.kos.txt')

In [13]:
output

{1: [['gop'],
  ['candidate'],
  ['races'],
  ['american'],
  ['years'],
  ['win'],
  ['lead'],
  ['november'],
  ['voters'],
  ['oct'],
  ['party'],
  ['bushs'],
  ['national'],
  ['states'],
  ['race'],
  ['news'],
  ['vote'],
  ['political'],
  ['administration'],
  ['john'],
  ['republican'],
  ['elections'],
  ['iraq'],
  ['state'],
  ['polls'],
  ['campaign'],
  ['senate'],
  ['president'],
  ['people'],
  ['primary'],
  ['media'],
  ['election'],
  ['republicans'],
  ['democrats'],
  ['time'],
  ['house'],
  ['poll'],
  ['war'],
  ['democratic'],
  ['kerry'],
  ['general'],
  ['bush']],
 2: [['candidate', 'democratic'],
  ['races', 'november'],
  ['races', 'polls'],
  ['races', 'senate'],
  ['races', 'primary'],
  ['races', 'media'],
  ['races', 'election'],
  ['races', 'republicans'],
  ['races', 'democrats'],
  ['races', 'house'],
  ['races', 'poll'],
  ['races', 'war'],
  ['races', 'democratic'],
  ['win', 'democratic'],
  ['lead', 'polls'],
  ['lead', 'poll'],
  ['lead', 'de

In [0]:
with open('output.txt', 'w') as file:
   file.write(str(output))


In [0]:
ls

docword.enron.txt  kos_.txt    vocab.enron.txt
docword.kos.txt    nips_.txt   vocab.kos.txt
docword.nips.txt   output.txt  vocab.nips.txt
